In [1]:
from z3 import *

In [ ]:
components={}
variables={}
constraints={}

In [2]:
s=Solver()

l_inhibit=0
l_up=1
l_down=2
l_c=3
l_x,l_y=Ints('l_x l_y')

inhibit=[1,1,1]
up=[0,11,-20]
down=[100,110,60]
c=Int('c')
x=[]
y=[]
r=[]
for i in range(3):
    x.append(If(l_x==l_inhibit,inhibit[i],If(l_x==l_up,up[i],If(l_x==l_down,down[i],c))))
    y.append(If(l_y==l_inhibit,inhibit[i],If(l_y==l_up,up[i],If(l_y==l_down,down[i],c))))
    r.append(x[i]+y[i])

s.add(l_x>=0,l_x<4,l_y>=0,l_y<4)
s.add(r[0]<=100,r[1]>110,r[2]>60)
print(s.check())
print(s.model())

sat
[c = 100, l_x = 3, l_y = 1]


In [8]:
s=Solver()
x=[4,5,6,8]
c=Int('c')
s.add(x[0]<c,x[1]<c,Not(x[2]<c),Not(x[3]<c))
print(s.check())
print(s.model())

sat
[c = 6]


In [9]:
s=Solver()
x=[4,5,6,8]
c=Int('c')
s.add(x[0]==c,x[1]==c,Not(x[2]==c),Not(x[3]==c))
print(s.check())

unsat


Z3Exception: model is not available